In [1]:
import wget

In [3]:
wget.download('http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py')

'bm_preproc.py'

In [4]:
wget.download('http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta')

'chr1.GRCh38.excerpt.fasta'

In [22]:
from bm_preproc import *

In [17]:
def read_FAST_A(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return(genome)

dna = read_FAST_A('chr1.GRCh38.excerpt.fasta')

In [15]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [25]:
def boyer_moore(p, p_bm, t):
    i = 0
    comp = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                comp += 1
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return comp, occurrences

In [26]:
p_bm = BoyerMoore(p)

In [27]:
boyer_moore(p, p_bm, dna)

(127973, [56922])

In [32]:
def naive(p, t):
    comp = 0
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            comp += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return comp, occurrences

In [33]:
naive(p, dna)

(984143, [56922])

In [34]:
wget.download('https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py')

'kmer_index.py'

In [35]:
from kmer_index import *

In [48]:
def ph(p, index, k):
    __init__(index, dna, k)
    hits = set(query(index, p))
    return hits

In [49]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [66]:
def queryIndex_approximate_match(p, t, n, index):
    
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    indexhits = 0
    for i in range(n+1):
        # split p into n+1 segments
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        indexhits += 1
        
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    print("indexhits = {}", indexhits)
    return list(all_matches)

In [67]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [68]:
index = Index(dna,8)
print(len(queryIndex_approximate_match(p, dna, 2, index)))

indexhits = {} 3
19


In [69]:
print(len(index.query(p)))

13


In [58]:
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [59]:
index = SubseqIndex(dna, 8, 3)
print(len(queryIndex_approximate_match(p, dna, 2, index)))

indexhits = {} 3
0


In [70]:
def approximate_match_subseq(p, t, n, ival):
    segment_length = int(round(len(p) / (n + 1)))
    all_matches = set()
    p_idx = SubseqIndex(t, segment_length, ival)
    idx_hits = 0
    for i in range(n + 1):
        start = i
        matches = p_idx.query(p[start:])

        # Extend matching segments to see if whole p matches
        for m in matches:
            idx_hits += 1
            if m < start or m - start + len(p) > len(t):
                continue

            mismatches = 0

            for j in range(0, len(p)):
                if not p[j] == t[m - start + j]:
                    mismatches += 1
                    if mismatches > n:
                        break

            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), idx_hits

In [72]:
print(approximate_match_subseq(p, dna, 2, 3))

([273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299], 79)
